![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [86]:
 # Start your code here!
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from data import CustomerAnalytics
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from data import BuildModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
repo = CustomerAnalytics(wrangled=True)
df = repo.wrangle()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2015 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2015 non-null   int32   
 1   city                    2015 non-null   category
 2   city_development_index  2015 non-null   float16 
 3   gender                  2015 non-null   category
 4   relevant_experience     2015 non-null   category
 5   enrolled_university     2015 non-null   category
 6   education_level         2015 non-null   category
 7   major_discipline        2015 non-null   category
 8   experience              2015 non-null   category
 9   company_size            2015 non-null   category
 10  company_type            2015 non-null   category
 11  last_new_job            2015 non-null   category
 12  training_hours          2015 non-null   int32   
 13  job_change              2015 non-null   float16 
dtypes: category(10), float1

In [63]:
df.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,Other,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0.0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0.0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0.0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0.0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0.0


# Predicting that a student will not what to change jobs

In [64]:
# Naive Model
int(df['job_change'].value_counts(normalize=True)[1] * 100)

df.nunique()
cols_to_drop = ['city', 'city_development_index', 'student_id', 'job_change']
target = 'job_change'
X = df.drop(columns=cols_to_drop)
y = df[target]

In [65]:
X.shape, y.shape

((2015, 10), (2015,))

In [85]:
repo.training_hrs_plot()